In [2]:
# import lib
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler

from keras.api.models import Sequential
from keras.api.layers import Dense
from keras.api.layers import LSTM
from keras.api.layers import Dropout

from keras.api.models import save_model

import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data.csv")
df.set_index("date")

,price,warehouse_capacity,truck_capacity
date,,,
2024-01-01,43.67,58,60
2024-01-02,83.13,80,41
2024-01-03,27.28,70,42
2024-01-04,66.99,57,45
2024-01-05,68.25,57,57
...,...,...,...
2024-04-05,80.92,66,59
2024-04-06,85.43,73,59
2024-04-07,44.13,72,54


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               100 non-null    float64
 1   warehouse_capacity  100 non-null    int64  
 2   truck_capacity      100 non-null    int64  
 3   date                100 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 3.3+ KB


In [4]:
sc = MinMaxScaler(feature_range=(0, 1))

In [7]:
sc.fit_transform(df[['price', 'warehouse_capacity', 'truck_capacity']])

array([[0.43095677, 0.26666667, 0.66666667],
       [0.83517722, 1.        , 0.03333333],
       [0.26306085, 0.66666667, 0.06666667],
       [0.66984225, 0.23333333, 0.16666667],
       [0.68274944, 0.23333333, 0.56666667],
       [0.90688384, 0.93333333, 0.23333333],
       [0.81827494, 0.6       , 0.9       ],
       [0.61852079, 0.86666667, 0.43333333],
       [0.94601516, 1.        , 0.83333333],
       [0.9563614 , 0.93333333, 0.33333333],
       [0.68223725, 0.4       , 0.26666667],
       [0.45666872, 0.6       , 0.36666667],
       [0.82544561, 0.63333333, 0.96666667],
       [0.17670559, 0.1       , 0.7       ],
       [0.35423069, 0.76666667, 0.5       ],
       [0.31315304, 0.43333333, 0.16666667],
       [0.4795124 , 0.53333333, 0.86666667],
       [0.98699037, 1.        , 0.6       ],
       [0.96291743, 0.66666667, 0.56666667],
       [0.93577136, 0.53333333, 0.93333333],
       [0.22536365, 0.23333333, 0.43333333],
       [0.53257529, 0.3       , 0.76666667],
       [0.

In [ ]:
df.info